# NLP Project, Classification of Amazon Reviews and Key Phrases
#### CSCI 3832 Natural Language Processing
Members: Adam Wuth, Benjamin Kohav, Noah Vilas, Aiden Devine, Evan Zachary

### Requirements

In [3]:
import os, random, sys, copy
import torch, torch.nn as nn, numpy as np
from tqdm.notebook import tqdm
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from nltk.tokenize import word_tokenize
from datasets import load_dataset, concatenate_datasets, load_from_disk
from datetime import datetime

### Load in the data set
The dataset is split into categories, but we wanted all categories from 2020 onwards. This code block will take forever to run, only run it the first time to get the dataset. 

In [4]:
# # #The dataset is split into categories

# categories = [
#     "All_Beauty",
#     "Amazon_Fashion",
#     "Appliances",
#     "Arts_Crafts_and_Sewing",
#     "Automotive",
#     "Baby_Products",
#     "Beauty_and_Personal_Care",
#     "Books",
#     "CDs_and_Vinyl",
#     "Cell_Phones_and_Accessories",
#     "Clothing_Shoes_and_Jewelry",
#     "Digital_Music",
#     "Electronics",
#     "Gift_Cards",
#     "Grocery_and_Gourmet_Food",
#     "Handmade_Products",
#     "Health_and_Household",
#     "Health_and_Personal_Care",
#     "Home_and_Kitchen",
#     "Industrial_and_Scientific",
#     "Kindle_Store",
#     "Magazine_Subscriptions",
#     "Movies_and_TV",
#     "Musical_Instruments",
#     "Office_Products",
#     "Patio_Lawn_and_Garden",
#     "Pet_Supplies",
#     "Software",
#     "Sports_and_Outdoors",
#     "Subscription_Boxes",
#     "Tools_and_Home_Improvement",
#     "Toys_and_Games",
#     "Video_Games",
#     "Unknown"
# ]

# # #to get reviews from 2023 onwards 2020 onwards was millions of reviews and was taking
# # #over an hour just to load the data
# # start_timestamp = int(datetime(2023, 1, 1).timestamp() * 1000)

# # #to store all datasets
# # allcats = []

# # for cat in categories:
# #     print(f"Loading category: {cat}")
# #     dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", f"raw_review_{cat}", split="full[:150000]",  trust_remote_code=True)
# #     #dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", f"raw_review_{cat}", split="full[:1%]",  trust_remote_code=True)
# #    #dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", cat, split="full",  trust_remote_code=True) formatting issues
# #     #get the 2023 onwards and add to data
# #     filtered_dataset = dataset.filter(lambda x: x['timestamp'] >= start_timestamp)
# #     #allcats.append(dataset)
# #     allcats.append(filtered_dataset)
# # #make one final dataset    
# # reviews = concatenate_datasets(allcats)


# limit = 589  # 100,000 target reviews 34 categories 5 stars, (100,000/34)/5 = 889

# allcats = []

# for cat in categories:
#     print(f"Loading category: {cat}")
#     #arbitrary 10000000 to make sure I get enough data after filter
#     dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", f"rawreview{cat}", split="full[:1000000]", trust_remote_code=True)
#     #dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", f"rawreview{cat}", split="full[:1%]",  trust_remote_code=True)
#     #to get reviews from 2023 onwards 2020 onwards was millions of reviews and was taking
#     #over an hour just to load the data
#     filtered_dataset = dataset.filter(lambda x: x['timestamp'] >= int(datetime(2023, 1, 1).timestamp() * 1000))
#     #in each category, for stars 1-5(not inclusive)
#     for star in range(1, 6):
#         data = dataset.filter(lambda x: int(float(x["rating"])) == star)
#         if len(data) >= limit:
#             #trim extra reviews randomly to avoid bias
#             data = data.shuffle().select(range(limit))
#             allcats.append(data)
# #Save reviews so we don't have to run code again
# reviews = concatenate_datasets(allcats)
# reviews.save_to_disk("filetred_amazon_reviews")
# print(Counter(reviews["rating"])) 
# print(f"Total reviews loaded: {len(reviews)}")
# Counter({1.0: 20026, 2.0: 20026, 3.0: 20026, 4.0: 20026, 5.0: 20026})



# reviews = load_from_disk("filtered_amazon_reviews")

# print(f"Total reviews loaded: {len(reviews)}")

# reviews.save_to_disk("filtered_amazon_reviews")

# print(Counter(reviews["rating"]))


If you have run that already, reviews was saved(should be in the working directory)so you can just do the next code block instead

In [5]:
from collections import Counter

reviews = load_from_disk("./filtered_amazon_reviews")

print(Counter(reviews["rating"]))


print(len(reviews))
print(reviews[0])
print(reviews[1])
print(reviews.column_names)

Counter({1.0: 20026, 2.0: 20026, 3.0: 20026, 4.0: 20026, 5.0: 20026})
100130
{'rating': 1.0, 'title': 'Worst nail polish ever', 'text': 'Worst nail polish ever! My daughter and I both used this nail polish in two different colors and now our nails are damaged. Our nails split horizontally and are peeling. Plus the damage has caused pain. Worst Sally Hansen product ever!', 'images': [], 'asin': 'B011855ADM', 'parent_asin': 'B011855ADM', 'user_id': 'AEMVAG56MA7MAFULCQJEOVJCKGHA', 'timestamp': 1454738837000, 'helpful_vote': 8, 'verified_purchase': True}
{'rating': 1.0, 'title': 'No funciona para mi', 'text': 'Bueno en cuanto a mi respondo que no me funciono. Tengo pocas pestañas, las enchufe antes de poner la máscara y el resultado desastroso. El producto hizo que mis pestañas perdieran el volumen del encrespado horrible.', 'images': [], 'asin': 'B09GTV6WL6', 'parent_asin': 'B09GTV6WL6', 'user_id': 'AFPNHXMEBYKO3SPMFXZCALLZ5IHA', 'timestamp': 1645820993736, 'helpful_vote': 5, 'verified_pu

### Load in the Glove Embeddings

In [1]:
import numpy as np
from nltk.tokenize import word_tokenize

from tqdm import tqdm

# ---- 1. Load GloVe Embeddings ----
glove_file = "glove.6B.50d.txt"  # Update path if needed
embeddings_dict = {}

print("Loading GloVe embeddings...")

with open(glove_file, 'r', encoding='utf8', errors="replace") as f:
    for line in f:
        line = line.strip().split(' ')
        word = line[0]
        embed = np.asarray(line[1:], dtype=np.float32)
        if embed.shape[0] != 50:
            continue
        embeddings_dict[word] = embed

print('Loaded {} words from GloVe'.format(len(embeddings_dict)))

# ---- 2. Build Word Embedding Matrix ----
embedding_dim = 50
embedding_matrix = []
word2id = {}

for i, word in enumerate(embeddings_dict):
    word2id[word] = i
    embedding_matrix.append(embeddings_dict[word])

# Add <unk> and <pad>
word2id['<unk>'] = len(word2id)
embedding_matrix.append(np.random.uniform(-0.1, 0.1, embedding_dim))

word2id['<pad>'] = len(word2id)
embedding_matrix.append(np.zeros(embedding_dim))

embedding_matrix = np.stack(embedding_matrix)

print("Unknown token index:", word2id['<unk>'])
print("Padding token index:", word2id['<pad>'])

# ---- 3. Build N-gram Embeddings (bi, tri, quad) ----

def build_ngram_vocab(reviews, n, embeddings_dict):
    ngram2id = {'<unk>': 0, '<pad>': 1}
    ngram_embeddings = [
        np.random.uniform(-0.1, 0.1, embedding_dim),  # <unk>
        np.zeros(embedding_dim)                      # <pad>
    ]

#     for text in reviews["text"]:
    for text in tqdm(reviews["text"], desc=f"Building {n}-grams"):
        tokens = word_tokenize(text.lower())
        for i in range(len(tokens) - n + 1):
            ngram_words = tokens[i:i+n]
            ngram = "_".join(ngram_words)
            if ngram not in ngram2id:
                vectors = [embeddings_dict.get(w, embeddings_dict.get('<unk>', np.random.uniform(-0.1, 0.1, embedding_dim))) for w in ngram_words]
                avg_embedding = np.mean(vectors, axis=0)
                ngram2id[ngram] = len(ngram2id)
                ngram_embeddings.append(avg_embedding)

    ngram_embedding_matrix = np.stack(ngram_embeddings)
    return ngram2id, ngram_embedding_matrix

print("\nBuilding bigram vocabulary and embeddings...")
bigram2id, bigram_embedding_matrix = build_ngram_vocab(reviews, n=2, embeddings_dict=embeddings_dict)
print("Bigram vocab size:", len(bigram2id))

print("\nBuilding trigram vocabulary and embeddings...")
trigram2id, trigram_embedding_matrix = build_ngram_vocab(reviews, n=3, embeddings_dict=embeddings_dict)

print("Trigram vocab size:", len(trigram2id))

print("\nBuilding quadgram vocabulary and embeddings...")
quadgram2id, quadgram_embedding_matrix = build_ngram_vocab(reviews, n=4, embeddings_dict=embeddings_dict)

print("Quadgram vocab size:", len(quadgram2id))


Loading GloVe embeddings...
Loaded 400000 words from GloVe
Unknown token index: 400000
Padding token index: 400001

Building bigram vocabulary and embeddings...


NameError: name 'reviews' is not defined

### Set up train and validation datasets

In [ ]:
import torch
from torch.utils.data import Dataset
from nltk.tokenize import word_tokenize

class NgramReviewDataset(Dataset):
    def __init__(self, reviews, labels, ngram2id, n=2, max_len=256):
        """
        Args:
            reviews: list of review texts
            labels: list of corresponding labels
            ngram2id: dictionary mapping n-gram strings to IDs
            n: size of the n-grams (e.g., 2 for bigrams, 3 for trigrams, etc.)
            max_len: maximum sequence length (padded or truncated)
        """
        self.reviews = reviews
        self.labels = labels
        self.ngram2id = ngram2id
        self.n = n
        self.max_len = max_len

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, idx):
        text = self.reviews[idx]
        label = self.labels[idx]

        tokens = word_tokenize(text.lower())

        # Generate n-grams
        ngrams = [tokens[i:i + self.n] for i in range(len(tokens) - self.n + 1)]

        # Convert n-grams to string format and map to IDs
        ngram_ids = []
        for ng in ngrams:
            ngram_str = "_".join(ng)
            ngram_id = self.ngram2id.get(ngram_str, self.ngram2id.get('<unk>', 0))
            ngram_ids.append(ngram_id)

        # Pad or truncate
        if len(ngram_ids) < self.max_len:
            pad_id = self.ngram2id.get('<pad>', 0)
            ngram_ids.extend([pad_id] * (self.max_len - len(ngram_ids)))
        else:
            ngram_ids = ngram_ids[:self.max_len]

        # Convert to tensors
        ngram_ids = torch.tensor(ngram_ids, dtype=torch.long)
        label = torch.tensor(label, dtype=torch.long)

        return ngram_ids, label

In [11]:
from sklearn.model_selection import train_test_split
from nltk.util import ngrams
def build_ngram_vocab(reviews, n=2, min_freq=2, limit=100000):
    counter = Counter()
    for review in reviews[:limit]:
        text = review['text'].lower() if isinstance(review, dict) else review.lower()
        tokens = word_tokenize(text)
        ngram_list = ngrams(tokens, n)
        ngram_strs = ["_".join(ng) for ng in ngram_list]
        counter.update(ngram_strs)

    # Filter by frequency
    vocab = {
        ng: idx + 1 for idx, (ng, count) in enumerate(counter.items()) if count >= min_freq
    }
    vocab['<pad>'] = 0
    vocab['<unk>'] = len(vocab)

    print(f"[n={n}] Total unique n-grams found: {len(counter)}")
    print(f"[n={n}] N-grams kept (freq ≥ {min_freq}): {len(vocab) - 2}")  # -2 to exclude special tokens
    return vocab


review_texts = [r['text'] if isinstance(r, dict) else r for r in reviews]
review_labels = [int(r['rating']) - 1 if isinstance(r, dict) else r for r in reviews]


train_texts, val_texts, train_labels, val_labels = train_test_split(
    review_texts, review_labels, test_size=0.1, random_state=42
)

bigram2id = build_ngram_vocab(train_texts, n=2, min_freq=2)
trigram2id = build_ngram_vocab(train_texts, n=3, min_freq=2)
quadgram2id = build_ngram_vocab(train_texts, n=4, min_freq=2)


train_bigram_dataset = NgramReviewDataset(train_texts, train_labels, bigram2id, n=2)
val_bigram_dataset = NgramReviewDataset(val_texts, val_labels, bigram2id, n=2)

train_trigram_dataset = NgramReviewDataset(train_texts, train_labels, trigram2id, n=3)
val_trigram_dataset = NgramReviewDataset(val_texts, val_labels, trigram2id, n=3)

train_quadgram_dataset = NgramReviewDataset(train_texts, train_labels, quadgram2id, n=4)
val_quadgram_dataset = NgramReviewDataset(val_texts, val_labels, quadgram2id, n=4)


print(f"Bigram Train: {len(train_bigram_dataset)} | Validation: {len(val_bigram_dataset)}")
print(f"Trigram Train: {len(train_trigram_dataset)} | Validation: {len(val_trigram_dataset)}")
print(f"Quadgram Train: {len(train_quadgram_dataset)} | Validation: {len(val_quadgram_dataset)}")


def vectorize_review_ngrams(text, ngram2id, n=2):
    tokens = word_tokenize(text.lower())
    ngram_list = ngrams(tokens, n)
    return [ngram2id.get("_".join(ng), ngram2id.get('<unk>', 0)) for ng in ngram_list]

[n=2] Total unique n-grams found: 121576
[n=2] N-grams kept (freq ≥ 2): 33178
[n=3] Total unique n-grams found: 248099
[n=3] N-grams kept (freq ≥ 2): 32146
[n=4] Total unique n-grams found: 310088
[n=4] N-grams kept (freq ≥ 2): 16694
Bigram Train: 6548 | Validation: 728
Trigram Train: 6548 | Validation: 728
Quadgram Train: 6548 | Validation: 728


In [19]:

class NgramClassifier(nn.Module):
    def __init__(self, ngram_matrix, ngram2id, num_classes):
        super(NgramClassifier, self).__init__()
        vocab_size, embed_dim = ngram_matrix.shape

        self.embedding = nn.Embedding.from_pretrained(
            torch.tensor(ngram_matrix, dtype=torch.float32),
            padding_idx=ngram2id['<pad>']
        )
        self.fc = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        embed = self.embedding(x).mean(dim=1)
        return self.fc(embed)
    
def create_ngram_classifiers(bigram_data, trigram_data, quadgram_data, num_classes):
    bigram_matrix, bigram2id = bigram_data
    trigram_matrix, trigram2id = trigram_data
    quadgram_matrix, quadgram2id = quadgram_data

    return {
        'bigram': NgramClassifier(bigram_matrix, bigram2id, num_classes),
        'trigram': NgramClassifier(trigram_matrix, trigram2id, num_classes),
        'quadgram': NgramClassifier(quadgram_matrix, quadgram2id, num_classes),
    }

In [20]:

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# Example constants (adjust as needed)
batch_size = 32
num_epochs = 5
learning_rate = 1e-4
num_classes = 5
max_len = 100  # example max length for padding

# Data encoding functions
def encode_ngrams(sentences, ngram2id, n, max_len):
    encoded = []
    for sent in sentences:
        indices = []
        for i in range(len(sent) - (n - 1)):
            ngram = "_".join(sent[i:i+n])
            idx = ngram2id.get(ngram, ngram2id['<unk>'])
            indices.append(idx)
        while len(indices) < max_len:
            indices.append(ngram2id['<pad>'])
        encoded.append(indices[:max_len])
    return torch.tensor(encoded)

# Use the factory and classifier from earlier
# create_ngram_classifiers(...) and NgramClassifier class assumed already defined

# Build datasets and loaders
def build_loader(x_tensor, y_tensor):
    dataset = TensorDataset(x_tensor, y_tensor)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

train_loaders = {
    'bigram': DataLoader(train_bigram_dataset, batch_size=batch_size, shuffle=True),
    'trigram': DataLoader(train_trigram_dataset, batch_size=batch_size, shuffle=True),
    'quadgram': DataLoader(train_quadgram_dataset, batch_size=batch_size, shuffle=True)
}

val_loaders = {
    'bigram': DataLoader(val_bigram_dataset, batch_size=batch_size),
    'trigram': DataLoader(val_trigram_dataset, batch_size=batch_size),
    'quadgram': DataLoader(val_quadgram_dataset, batch_size=batch_size)
}

# Create models
models = create_ngram_classifiers(
    (bigram_embedding_matrix, bigram2id),
    (trigram_embedding_matrix, trigram2id),
    (quadgram_embedding_matrix, quadgram2id),
    num_classes=num_classes
)

# Create separate optimizers & loss
criterions = {k: nn.CrossEntropyLoss() for k in models}
optimizers = {
    k: torch.optim.Adam(models[k].parameters(), lr=learning_rate)
    for k in models
}

# Evaluation function
def predict(model, dataloader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for batch_x, batch_y in dataloader:
            logits = model(batch_x)
            preds = torch.argmax(logits, dim=1)
            correct += (preds == batch_y).sum().item()
            total += batch_y.size(0)
    return correct / total

# Training loop for all models
for ngram in ['bigram', 'trigram', 'quadgram']:
    print(f"\n==== Training {ngram} model ====")
    model = models[ngram]
    optimizer = optimizers[ngram]
    criterion = criterions[ngram]
    train_loader = train_loaders[ngram]
    val_loader = val_loaders[ngram]

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        for batch_x, batch_y in train_loader:
            optimizer.zero_grad()
            logits = model(batch_x)
            loss = criterion(logits, batch_y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        val_acc = predict(model, val_loader)
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}, Val Acc: {val_acc * 100:.2f}%")

    final_acc = predict(model, val_loader)
    print(f"Final Validation Accuracy for {ngram}: {final_acc * 100:.2f}%")



==== Training bigram model ====
Epoch 1/5, Loss: 1.6196, Val Acc: 36.68%
Epoch 2/5, Loss: 1.5607, Val Acc: 63.60%
Epoch 3/5, Loss: 1.5077, Val Acc: 64.42%
Epoch 4/5, Loss: 1.4603, Val Acc: 64.42%
Epoch 5/5, Loss: 1.4182, Val Acc: 64.42%
Final Validation Accuracy for bigram: 64.42%

==== Training trigram model ====
Epoch 1/5, Loss: 1.6101, Val Acc: 23.63%
Epoch 2/5, Loss: 1.5472, Val Acc: 64.42%
Epoch 3/5, Loss: 1.4938, Val Acc: 64.42%
Epoch 4/5, Loss: 1.4489, Val Acc: 64.42%
Epoch 5/5, Loss: 1.4109, Val Acc: 64.42%
Final Validation Accuracy for trigram: 64.42%

==== Training quadgram model ====
Epoch 1/5, Loss: 1.5663, Val Acc: 62.77%
Epoch 2/5, Loss: 1.5071, Val Acc: 64.42%
Epoch 3/5, Loss: 1.4554, Val Acc: 64.42%
Epoch 4/5, Loss: 1.4114, Val Acc: 64.42%
Epoch 5/5, Loss: 1.3741, Val Acc: 64.42%
Final Validation Accuracy for quadgram: 64.42%


In [21]:
Counter(val_labels)

Counter({4: 469, 0: 90, 3: 78, 2: 48, 1: 43})